In [1]:
import torch

### 生成数据集与小批量样本

In [2]:
dim_features = 8
true_w = torch.normal(0, 4, (dim_features, ))
true_b = torch.normal(4, 1, (1, ))
true_w, true_b

(tensor([-6.2018,  0.7568,  0.7294, -4.1338,  6.0404,  1.0031, -1.5603,  9.7739]),
 tensor([3.0905]))

In [3]:
def synthetic_data(true_w, true_b, num_examples):
    dim_features = true_w.shape[0]
    X = torch.normal(0, 1, (num_examples, dim_features))
    y = torch.mv(X, true_w) + true_b
    y += torch.normal(0, 0.01, (num_examples, ))
    return X, y.reshape((-1, 1))

num_examples = 400
features, labels = synthetic_data(true_w, true_b, num_examples)
features.shape, labels.shape

(torch.Size([400, 8]), torch.Size([400, 1]))

In [4]:
from torch.utils import data

def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, is_train)

batch_size = 20
data_iter = load_array((features, labels), batch_size)

### 定义模型与参数

In [5]:
from torch import nn
net = nn.Sequential(nn.Linear(dim_features, 1))
net[0].weight.data.normal_(1, 0.1), net[0].bias.data.normal_(0, 0.1)

(tensor([[0.8735, 1.0928, 0.9915, 1.0477, 0.9721, 1.0611, 0.7159, 1.0733]]),
 tensor([0.1120]))

### 定义损失函数与优化算法

In [6]:
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.05)

### 训练模型并检验正确性

In [7]:
num_epochs = 5
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}: loss = {l:f}')

w = net[0].weight.data
b = net[0].bias.data
w - true_w, b - true_b

epoch 1: loss = 2.889395
epoch 2: loss = 0.050088
epoch 3: loss = 0.001043
epoch 4: loss = 0.000109
epoch 5: loss = 0.000098


(tensor([[ 8.5831e-06, -6.1047e-04,  2.9814e-04, -1.0266e-03, -2.5463e-04,
           1.0371e-04,  6.3086e-04, -7.3814e-04]]),
 tensor([5.3406e-05]))